In [16]:
# Importing packages

import numpy as np
import math as m
from scipy.spatial import ConvexHull
import os
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
import itertools

import timeit

%matplotlib inline

pd.set_option('display.max_columns', None)
#sn.set_palette("pastel")

In [17]:
# Load files

# BH+MS database
os.chdir('C:\\Users\\Utente\\Desktop\\data\\sevn_output_Z0.01A1L1')
file_int = pd.read_parquet('int_binsys.parquet')
file_nint = pd.read_parquet('non_int_binsys.parquet')

# Real data database (4 confirmed systems)
real_data = pd.read_parquet('real_data.parquet')

# Interaction type database
os.chdir('C:\\Users\\Utente\\Desktop\\data\\sevn_output_Z0.01A1L1')
type_df = pd.read_csv('type_df.csv')

bwt = pd.read_parquet('df_bwt.parquet')

In [18]:
file_nint.tail(10)

,ID,name,Mass_0,MHE_0,MCO_0,Radius_0,RHE_0,RCO_0,Luminosity_0,Temperature_0,Lambda_0,Phase_0,PhaseBSE_0,RemnantType_0,Zams_0,Mass_1,MHE_1,MCO_1,Radius_1,RHE_1,RCO_1,Luminosity_1,Temperature_1,Lambda_1,Phase_1,PhaseBSE_1,RemnantType_1,Zams_1,Semimajor,Eccentricity,GWtime,BWorldtime,BEvent,Period,logP
218045,933109,0_116269807422399,29.287180,0.0,0.0,0.000124,0.0,0.0,1.000000e-10,1637.000,NaN,7,14.0,6,45.84113,29.88802,0.0,0.0,10.706040,0.0,0.0,191046.20,36880.50,NaN,1,1.0,0,30.53500,8098.19600,0.344690,7.899538e+12,3.547908,0,10973.770474,4.040356
218320,933129,0_810873351414231,20.489000,0.0,0.0,0.000087,0.0,0.0,1.000000e-10,1957.163,NaN,7,14.0,6,42.16400,12.27492,0.0,0.0,4.696199,0.0,0.0,13265.34,28584.65,NaN,1,1.0,0,12.26200,28139.51000,0.275828,8.581308e+15,4.702707,0,95525.689595,4.980120
218428,933137,0_404457532291907,13.902600,0.0,0.0,0.000059,0.0,0.0,1.000000e-10,2375.962,NaN,7,14.0,6,20.99941,21.71303,0.0,0.0,7.955467,0.0,0.0,82164.07,34646.89,NaN,1,1.0,0,21.90459,774.02930,0.014039,5.011228e+09,4.291288,0,417.983463,2.621159
218486,933141,0_560096302193912,22.385240,0.0,0.0,0.000095,0.0,0.0,1.000000e-10,1872.434,NaN,7,14.0,6,34.54561,47.25368,0.0,0.0,31.275720,0.0,0.0,643348.50,29230.38,NaN,1,1.0,0,51.91834,383.39410,0.007180,4.404876e+07,3.547235,0,104.204251,2.017885
218487,933141,0_560096302193912,22.420110,0.0,0.0,0.000095,0.0,0.0,1.000000e-10,1870.978,NaN,7,14.0,6,34.54561,40.37604,0.0,0.0,182.926100,0.0,0.0,700406.70,12346.00,NaN,1,1.0,0,51.91834,398.79890,0.000000,6.683502e+07,3.993449,4,116.414676,2.066008
218556,933146,0_860933608483194,7.481464,0.0,0.0,0.000032,0.0,0.0,1.000000e-10,3238.875,NaN,7,14.0,6,10.97220,33.97683,0.0,0.0,35.490900,0.0,0.0,376801.70,24004.70,NaN,1,1.0,0,36.24764,75.19402,0.012060,4.554129e+05,6.003848,0,11.730393,1.069313
218557,933146,0_860933608483194,7.483689,0.0,0.0,0.000032,0.0,0.0,1.000000e-10,3238.393,NaN,7,14.0,6,10.97220,33.74358,0.0,0.0,39.272980,0.0,0.0,383204.70,22915.92,NaN,1,1.0,0,36.24764,71.03921,0.000000,3.674376e+05,6.062114,4,10.801852,1.033498
218944,933176,0_959328809051248,28.480180,0.0,0.0,0.000121,0.0,0.0,1.000000e-10,1660.031,NaN,7,14.0,6,44.47767,32.31217,0.0,0.0,12.165980,0.0,0.0,238778.60,36580.70,NaN,1,1.0,0,33.23700,6832.76900,0.486077,2.231608e+12,3.599397,0,8391.004533,3.923814
219812,933243,0_174462992539636,20.132570,0.0,0.0,0.000085,0.0,0.0,1.000000e-10,1974.412,NaN,7,14.0,6,24.65100,21.36253,0.0,0.0,16.772050,0.0,0.0,124876.70,26494.39,NaN,1,1.0,0,21.75500,20313.62000,0.504870,5.011168e+14,7.188198,0,52062.604699,4.716526
221024,933333,0_679432517052656,14.661950,0.0,0.0,0.000062,0.0,0.0,1.000000e-10,2313.618,NaN,7,14.0,6,22.18416,14.81684,0.0,0.0,5.246191,0.0,0.0,23034.46,31045.53,NaN,1,1.0,0,14.82459,221.01090,0.016961,5.590183e+07,4.178780,0,70.098549,1.845709


In [19]:
# Restrict data to T, P, M, e, a, L

restr_data_int = file_int[['ID', 'name', 'Radius_0', 'Mass_0', 'Temperature_0', 'Luminosity_0', 'Radius_1', 'Mass_1', 'Temperature_1', 'Luminosity_1', 'Eccentricity', 'Period', 'logP', 'Semimajor', 'BWorldtime']]
restr_data_nint = file_nint[['ID', 'name', 'Radius_0', 'Mass_0', 'Temperature_0', 'Luminosity_0', 'Radius_1', 'Mass_1', 'Temperature_1', 'Luminosity_1', 'Eccentricity', 'Period', 'logP', 'Semimajor', 'BWorldtime']]

# Rename *_0 as *_BH

restr_data_int = restr_data_int.rename(columns={'Radius_0': 'Radius_BH', 'Mass_0': 'Mass_BH', 'Temperature_0': 'Temperature_BH', 'Luminosity_0': 'Luminosity_BH'})
restr_data_nint = restr_data_nint.rename(columns={'Radius_0': 'Radius_BH', 'Mass_0': 'Mass_BH', 'Temperature_0': 'Temperature_BH', 'Luminosity_0': 'Luminosity_BH'})

# merging the BH+MS database with the type database
restr_data_nint = restr_data_nint.merge(type_df, on=['name'], how='left')

restr_data_nint = restr_data_nint.merge(bwt[['ID', 'name', 'elapsed_bwt']], on=['ID','name'], how='left')

# removing unwanted column
restr_data_nint = restr_data_nint.drop(columns=['Unnamed: 0'])

restr_data_nint = restr_data_nint.drop_duplicates(subset=['ID', 'name'], keep='first')

#print(np.shape(file_nint))
#print(restr_data_nint['type'].value_counts())

In [20]:
restr_data_nint

,ID,name,Radius_BH,Mass_BH,Temperature_BH,Luminosity_BH,Radius_1,Mass_1,Temperature_1,Luminosity_1,Eccentricity,Period,logP,Semimajor,BWorldtime,BEvent,type,elapsed_bwt
0,8,0_161360954723734,0.000081,19.132070,2025.380,1.000000e-10,6.372025,14.29574,29333.78,27084.51,0.065674,80274.801528,4.904579,25226.67000,7.610892,-1,NI,4.973328
1,10,0_379526196000863,0.000034,7.949073,3142.167,1.000000e-10,5.897774,18.29035,33853.44,41160.55,0.019055,15.227636,1.182632,76.82647,6.025122,-1,MT,6.465338
2,15,0_788916259432216,0.000082,19.255620,2018.872,1.000000e-10,6.138398,15.43642,30792.58,30520.23,0.487635,50989.914268,4.707484,18872.84000,5.720307,-1,NI,5.694563
3,52,0_209066710874162,0.000064,15.024700,2285.518,1.000000e-10,9.074868,25.10178,35801.70,121895.90,0.012461,120.663362,2.081575,351.80210,4.131395,-1,CE,2.505124
4,57,0_320500128815113,0.000146,34.442380,1509.527,1.000000e-10,11.635250,33.46420,37757.77,247896.40,0.034553,4627.164777,3.665315,4767.43800,3.263417,-1,NI,1.739941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53389,933141,0_560096302193912,0.000095,22.385240,1872.434,1.000000e-10,31.275720,47.25368,29230.38,643348.50,0.007180,104.204251,2.017885,383.39410,3.547235,-1,MT,0.507361
53391,933146,0_860933608483194,0.000032,7.481464,3238.875,1.000000e-10,35.490900,33.97683,24004.70,376801.70,0.012060,11.730393,1.069313,75.19402,6.003848,-1,MT,0.147293
53393,933176,0_959328809051248,0.000121,28.480180,1660.031,1.000000e-10,12.165980,32.31217,36580.70,238778.60,0.486077,8391.004533,3.923814,6832.76900,3.599397,-1,NI,1.544855
53394,933243,0_174462992539636,0.000085,20.132570,1974.412,1.000000e-10,16.772050,21.36253,26494.39,124876.70,0.504870,52062.604699,4.716526,20313.62000,7.188198,-1,NI,0.371452


In [21]:
# Removing duolicated line for the same id, jff

dupl_int = restr_data_int.drop_duplicates(subset=['ID'], keep='first')
dupl_nint = restr_data_nint.drop_duplicates(subset=['ID'], keep='first')

# Classification algos

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data into a Pandas DataFrame
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint[['type']]

# Encode the labels if they are not already encoded
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels.values.ravel())

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Define a list of classifiers
classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    GaussianNB(),
    LogisticRegression(),
    GradientBoostingClassifier(),
    AdaBoostClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
    ExtraTreesClassifier(),
    BaggingClassifier()
]

# Create empty lists to store evaluation metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Train and evaluate each classifier
for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    # Print evaluation metrics
    print(f"Classifier: {classifier_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("----------------------------------")

Classifier: DecisionTreeClassifier
Accuracy: 0.9698
Precision: 0.9699
Recall: 0.9698
F1-Score: 0.9698
----------------------------------
Classifier: RandomForestClassifier
Accuracy: 0.9789
Precision: 0.9791
Recall: 0.9789
F1-Score: 0.9789
----------------------------------


C:\Users\Utente\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Classifier: KNeighborsClassifier
Accuracy: 0.9597
Precision: 0.9600
Recall: 0.9597
F1-Score: 0.9597
----------------------------------
Classifier: GaussianNB
Accuracy: 0.8976
Precision: 0.8988
Recall: 0.8976
F1-Score: 0.8976
----------------------------------


C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classifier: LogisticRegression
Accuracy: 0.9212
Precision: 0.9205
Recall: 0.9212
F1-Score: 0.9201
----------------------------------
Classifier: GradientBoostingClassifier
Accuracy: 0.9720
Precision: 0.9722
Recall: 0.9720
F1-Score: 0.9721
----------------------------------
Classifier: AdaBoostClassifier
Accuracy: 0.7463
Precision: 0.8035
Recall: 0.7463
F1-Score: 0.7568
----------------------------------
Classifier: XGBClassifier
Accuracy: 0.9787
Precision: 0.9789
Recall: 0.9787
F1-Score: 0.9787
----------------------------------
Classifier: MLPClassifier
Accuracy: 0.9726
Precision: 0.9732
Recall: 0.9726
F1-Score: 0.9728
----------------------------------
Classifier: SVC
Accuracy: 0.9425
Precision: 0.9431
Recall: 0.9425
F1-Score: 0.9410
----------------------------------
Classifier: ExtraTreesClassifier
Accuracy: 0.9791
Precision: 0.9794
Recall: 0.9791
F1-Score: 0.9792
----------------------------------
Classifier: BaggingClassifier
Accuracy: 0.9753
Precision: 0.9756
Recall: 0.9753
F1-S

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import accuracy_score

# Load the data
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint['type']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create a Bayesian Gaussian Mixture classifier
classifier = BayesianGaussianMixture(n_components=10)  # You can adjust the number of components as needed

# Train the classifier
classifier.fit(X_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Assign class labels to predicted clusters
cluster_labels = {}
for cluster in range(len(classifier.means_)):
    cluster_labels[cluster] = y_train[y_train == cluster].value_counts().idxmax()

y_pred_labeled = [cluster_labels[label] for label in y_pred]

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred_labeled)
print("Accuracy:", accuracy)


C:\Users\Utente\anaconda3\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


ValueError: attempt to get argmax of an empty sequence

In [ ]:
!pip install pgmpy

In [25]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Assuming you have already loaded the data into 'data' and 'labels'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Rename the columns in 'X_train' to match the variable names in the BayesianModel
X_train.columns = ['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']

# Impute missing values with mean imputation
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Create a Bayesian Network model
model = BayesianModel([('Mass_BH', 'type'), ('Mass_1', 'type'), ('logP', 'type'), ('Eccentricity', 'type')])

# Estimate the parameters using Maximum Likelihood Estimation (MLE)
estimator = BayesianEstimator(model, X_train_imputed)
model.fit(X_train_imputed, estimator=estimator)

# Predict the labels for the test set
X_test_imputed = imputer.transform(X_test)
predictions = model.predict(X_test_imputed)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, predictions['type'])
print("Accuracy:", accuracy)

C:\Users\Utente\anaconda3\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have already loaded the data into 'data' and 'labels'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create a Bayesian Network model
model = BayesianModel([('Mass_BH', 'type'), ('Mass_1', 'type'), ('logP', 'type'), ('Eccentricity', 'type')])

# Estimate the parameters using Maximum Likelihood Estimation (MLE)
estimator = BayesianEstimator(model, X_train)
model.fit(X_train, estimator=estimator)

# Predict the labels for the test set
predictions = model.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, predictions['type'])
print("Accuracy:", accuracy)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import accuracy_score

# Load the data
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint['type']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create a Bayesian Gaussian Mixture classifier
classifier = BayesianGaussianMixture(n_components=10)  # You can adjust the number of components as needed

# Train the classifier
classifier.fit(X_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Load the data into a Pandas DataFrame
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint['type']

# Encode the string labels to numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Create and train the XGBoost classifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)

# Predict on the validation set
y_pred = xgb_classifier.predict(X_val)

# Plot the confusion matrix
plot_confusion_matrix(xgb_classifier, X_val, y_val)
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix, precision_recall_curve, roc_curve, auc
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Load the data into a Pandas DataFrame
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint['type']

# Encode the string labels to numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Create and train the XGBoost classifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)

# Predict probabilities on the validation set
y_pred_proba = xgb_classifier.predict_proba(X_val)

# Binarize the labels
y_val_binarized = label_binarize(y_val, classes=np.unique(y_val))

# Compute precision and recall for each class
precision = dict()
recall = dict()
n_classes = y_val_binarized.shape[1]
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_val_binarized[:, i], y_pred_proba[:, i])

# Plot the precision-recall curves for each class
plt.figure(figsize=(10, 6))
colors = ['blue', 'red', 'green']  # Customize colors as per the number of classes
for i, color in zip(range(n_classes), colors):
    plt.plot(recall[i], precision[i], color=color, lw=2, label='Class {}'.format(i))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

# Plot the confusion matrix
plot_confusion_matrix(xgb_classifier, X_val, y_val)
plt.title('Confusion Matrix')
plt.show()

# Plot the ROC curve
fpr, tpr, _ = roc_curve(y_val_binarized.ravel(), y_pred_proba.ravel())
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Classifier': [classifier.__class__.__name__ for classifier in classifiers],
    'Accuracy': accuracy_scores
})

# Plot the training and validation accuracies
plt.figure(figsize=(12, 6))
sns.barplot(data=results_df, x='Classifier', y='Accuracy', palette='viridis')
plt.xticks(rotation=90)
plt.xlabel('Classifier')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracies for each Classifier')
plt.tight_layout()
plt.show()

In [ ]:
# Create empty lists to store training and validation accuracy
training_accuracy = []
validation_accuracy = []

# Train and evaluate each classifier
for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    classifier.fit(X_train, y_train)
    y_train_pred = classifier.predict(X_train)
    y_pred = classifier.predict(X_test)
    
    training_acc = accuracy_score(y_train, y_train_pred)
    validation_acc = accuracy_score(y_test, y_pred)
    
    training_accuracy.append(training_acc)
    validation_accuracy.append(validation_acc)
    
    # Print evaluation metrics
    print(f"Classifier: {classifier_name}")
    print(f"Training Accuracy: {training_acc:.4f}")
    print(f"Validation Accuracy: {validation_acc:.4f}")
    print("----------------------------------")

# Plot the difference between training and validation accuracy for each classifier
plt.figure(figsize=(12, 8))
sns.barplot(x=classifiers, y=[train - val for train, val in zip(training_accuracy, validation_accuracy)])
plt.xticks(rotation=45)
plt.xlabel('Classifier')
plt.ylabel('Difference in Accuracy (Training - Validation)')
plt.title('Difference between Training and Validation Accuracy for each Classifier')
plt.tight_layout()
plt.show()

In [ ]:
# Create a DataFrame to compare the performance of each classifier
results_df = pd.DataFrame({
    'Classifier': [classifier.__class__.__name__ for classifier in classifiers],
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1-Score': f1_scores
})

# Plot comparison for Accuracy
plt.figure(figsize=(10, 6))
sns.barplot(x='Accuracy', y='Classifier', data=results_df.sort_values(by='Accuracy', ascending=False))
plt.title("Accuracy Comparison")
plt.xlabel("Accuracy")
plt.ylabel("Classifier")

# Plot comparison for Precision
plt.figure(figsize=(10, 6))
sns.barplot(x='Precision', y='Classifier', data=results_df.sort_values(by='Precision', ascending=False))
plt.title("Precision Comparison")
plt.xlabel("Precision")
plt.ylabel("Classifier")

# Plot comparison for Recall
plt.figure(figsize=(10, 6))
sns.barplot(x='Recall', y='Classifier', data=results_df.sort_values(by='Recall', ascending=False))
plt.title("Recall Comparison")
plt.xlabel("Recall")
plt.ylabel("Classifier")

# Plot comparison for F1-Score
plt.figure(figsize=(10, 6))
sns.barplot(x='F1-Score', y='Classifier', data=results_df.sort_values(by='F1-Score', ascending=False))
plt.title("F1-Score Comparison")
plt.xlabel("F1-Score")
plt.ylabel("Classifier")

# Find the best performing algorithm based on the metrics
best_accuracy_classifier = results_df.loc[results_df['Accuracy'].idxmax()]['Classifier']
best_precision_classifier = results_df.loc[results_df['Precision'].idxmax()]['Classifier']
best_recall_classifier = results_df.loc[results_df['Recall'].idxmax()]['Classifier']
best_f1_score_classifier = results_df.loc[results_df['F1-Score'].idxmax()]['Classifier']

print(f"Best Performing Algorithm (Accuracy): {best_accuracy_classifier}")
print(f"Best Performing Algorithm (Precision): {best_precision_classifier}")
print(f"Best Performing Algorithm (Recall): {best_recall_classifier}")
print(f"Best Performing Algorithm (F1-Score): {best_f1_score_classifier}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load the data into a Pandas DataFrame
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint[['type']]

# Encode the labels if they are not already encoded
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels.values.ravel())

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Normalize the input features
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

# Reshape the input data for CNN
X_train = X_train.values.reshape(-1, 2, 2, 1)
X_test = X_test.values.reshape(-1, 2, 2, 1)

# Define the CNN architecture
model = Sequential()
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(2, 2, 1)))
model.add(Conv2D(32, kernel_size=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred_classes = np.round(y_pred).flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Create a DataFrame to store the metrics
results_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Score': [accuracy, precision, recall, f1]
})

# Plot the metrics
plt.figure(figsize=(10, 6))
sns.barplot(x='Metric', y='Score', data=results_df)
plt.title("CNN Metrics")
plt.xlabel("Metric")
plt.ylabel("Score")


# Plot the training and validation accuracy over epochs
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

## Rescaling

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Rescale the selected columns
data[columns_to_rescale] = scaler.fit_transform(data[columns_to_rescale])

# Load the data into a Pandas DataFrame
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint[['type']] 

# Select the columns to rescale
columns_to_rescale = ['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']

### Standar scaler

In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Rescale the selected columns
data[columns_to_rescale] = scaler.fit_transform(data[columns_to_rescale])

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# One-hot encode the labels
#y_train = pd.get_dummies(y_train)
#y_test = pd.get_dummies(y_test)

### MinMax scaler

In [ ]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Rescale the selected columns
data[columns_to_rescale] = scaler.fit_transform(data[columns_to_rescale])

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# One-hot encode the labels
#y_train = pd.get_dummies(y_train)
#y_test = pd.get_dummies(y_test)

## SVM test

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Define the SVM classifier
clf = SVC(kernel='linear')

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

## Decision tree test

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Define the decision tree classifier
clf = DecisionTreeClassifier()

# Define the hyperparameter grid to search over
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Get the best classifier from the grid search
best_clf = grid_search.best_estimator_

# Fit the best classifier on the training data
best_clf.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = best_clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

## NN test

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Load the data into a Pandas DataFrame
data = restr_data_nint[['Mass_BH', 'Mass_1', 'logP', 'Eccentricity']]
labels = restr_data_nint[['type']] 

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# One-hot encode the labels
#y_train = pd.get_dummies(y_train)
#y_test = pd.get_dummies(y_test))

# Define the neural network classifier
clf = MLPClassifier()

# Define the hyperparameters to search
parameters = {'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)],
              'activation': ['relu', 'tanh', 'logistic'],
              'solver': ['lbfgs', 'adam', 'sgd'],
              'learning_rate': ['constant', 'invscaling', 'adaptive']}

# Create the GridSearchCV object
grid_search = GridSearchCV(clf, parameters, cv=5)

# Fit the GridSearchCV object on the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Predict the labels for the test data using the best model
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))
